In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb

import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 30)

#sns.set_style("whitegrid")
#plt.style.use('bmh')
plt.style.use('seaborn-whitegrid')

# this allows plots to appear directly in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
train_df = pd.read_csv('data/train.csv')
train_df['data_set'] = 'train'
test_df = pd.read_csv('data/test.csv')
test_df['data_set'] = 'test'
# combine train and test data into one df
test_df['registered'] = 0
test_df['casual'] = 0
test_df['count'] = 0

all_df = pd.concat([train_df, test_df])
# parse datetime colum & add new time related columns
dt = pd.DatetimeIndex(all_df['datetime'])
all_df.set_index(dt, inplace=True)

# logarithmic transformation of dependent cols
# (adding 1 first so that 0 values don't become -inf)
for col in ['casual', 'registered', 'count']:
    all_df[f'{col}_log'] = np.log(all_df[col] + 1)

all_df['date'] = dt.date # yyyymmdd
all_df['day'] = dt.day # dd
all_df['month'] = dt.month # mm
all_df['year'] = dt.year # yyyy
all_df['hour'] = dt.hour # hh
all_df['dow'] = dt.dayofweek #曜日 Mon:0 Tue:1 Wed:2 Thu:3 Fri:4 Sat:5 Sun:6
all_df['woy'] = dt.isocalendar().week #その日の週が年間で見ると何番目の週かを表す数字 [dt.weekofyear]は deprecated

# add a count_season column using join
by_season = all_df[all_df['data_set'] == 'train'].copy().groupby(['season'])[['count']].agg(sum)
by_season.columns = ['count_season']
all_df = all_df.join(by_season, on='season')


# feature engineer a new column whether its a peak hour or not
all_df['peak'] = all_df[['hour', 'workingday']]\
    .apply(lambda df: 3 if ((df['workingday'] == 1 and (df['hour'] == 8 or 17 <= df['hour'] <= 18)) \
                            or (df['workingday'] == 0 and 11 <= df['hour'] <= 17)) else \
                            ( 2 if ((df['workingday'] == 1 and (df['hour'] == 7 or df['hour'] == 9 or df['hour'] == 16 or 19 <= df['hour'] <= 20)) \
                            or (df['workingday'] == 0 and (df['hour'] == 10 or 18 <= df['hour'] <= 19))) else \
                            ( 1 if ((df['workingday'] == 1 and (10 <= df['hour'] <= 15 or 21 <= df['hour'] <= 22)) \
                            or (df['workingday'] == 0 and (8 <= df['hour'] <= 9 or 20 <= df['hour'] <= 23))) else 0)), axis = 1)

#ここの修正の仕方は、間違っているので要修正！
# sandy
#all_df['holiday'] = all_df[['month', 'day', 'holiday', 'year']]\
#    .apply(lambda df: 1 if (df['year'] == 2012 and df['month'] == 10 and df['day'] == 30) else 0, axis = 1)
# 修正後↓
all_df['holiday'] = all_df[['month', 'day', 'holiday', 'year']]\
    .apply(lambda df: 1 if (df['year'] == 2012 and df['month'] == 10 and df['day'] == 30) else df['holiday'], axis = 1)


# christmas and others
all_df['holiday'] = all_df[['month', 'day', 'holiday']]\
    .apply(lambda df: 1 if (df['month'] == 12 and df['day'] in [24, 26, 31]) else df['holiday'], axis = 1)
all_df['workingday'] = all_df[['month', 'day', 'workingday']]\
    .apply(lambda df: 0 if df['month'] == 12 and df['day'] in [24, 31] else df['workingday'], axis = 1)
# これは流石に気づかない気がする。。。気づけない気がする。。。
def get_day(day_start):
    day_end = day_start + pd.offsets.DateOffset(hours=23)
    return pd.date_range(day_start, day_end, freq="H")

# tax day
all_df.loc[get_day(datetime(2011, 4, 15)), "workingday"] = 1
all_df.loc[get_day(datetime(2012, 4, 16)), "workingday"] = 1

# thanksgiving friday
all_df.loc[get_day(datetime(2011, 11, 25)), "workingday"] = 0
all_df.loc[get_day(datetime(2012, 11, 23)), "workingday"] = 0

# tax day
all_df.loc[get_day(datetime(2011, 4, 15)), "holiday"] = 0
all_df.loc[get_day(datetime(2012, 4, 16)), "holiday"] = 0

# thanksgiving friday
all_df.loc[get_day(datetime(2011, 11, 25)), "holiday"] = 1
all_df.loc[get_day(datetime(2012, 11, 23)), "holiday"] = 1

#storms
all_df.loc[get_day(datetime(2012, 5, 21)), "holiday"] = 1

#tornado
all_df.loc[get_day(datetime(2012, 6, 1)), "holiday"] = 1
# from histogram
all_df['ideal'] = all_df[['temp', 'windspeed']]\
    .apply(lambda df: 1 if (df['temp'] > 27 and df['windspeed'] < 30) else 0, axis = 1)
    
all_df['sticky'] = all_df[['humidity', 'workingday']]\
    .apply(lambda df: 1 if (df['workingday'] == 1 and df['humidity'] >= 60) else 0, axis = 1)

# One-hot-Encoding for season
#season_map = {1:'Spring', 2:'Summer', 3:'Fall', 4:'Winter'}
#all_df['season'] = all_df['season'].map(lambda d : season_map[d])
#temporary = pd.get_dummies(all_df['season'])
#all_df['season_Fall'] = temporary['Fall']
#all_df['season_Spring'] = temporary['Spring']
#all_df['season_Summer'] = temporary['Summer']
#all_df['season_Winter'] = temporary['Winter']

# temperature
all_df.loc[all_df['temp'] < 10,'temp_cat'] = 1
all_df.loc[(all_df['temp'] >= 10) & (all_df['temp'] < 15),'temp_cat'] = 2
all_df.loc[(all_df['temp'] >= 15) & (all_df['temp'] < 20),'temp_cat'] = 3
all_df.loc[(all_df['temp'] >= 20) & (all_df['temp'] < 25),'temp_cat'] = 4
all_df.loc[(all_df['temp'] >= 25) & (all_df['temp'] < 30),'temp_cat'] = 5
all_df.loc[(all_df['temp'] >= 30) & (all_df['temp'] < 35),'temp_cat'] = 6
all_df.loc[(all_df['temp'] >= 35),'temp_cat'] = 7

# humidity
#all_df.loc[all_df['humidity'] < 10,'humidity_cat'] = 0
#all_df.loc[(all_df['humidity'] >= 10) & (all_df['humidity'] < 20),'humidity_cat'] = 1
#all_df.loc[(all_df['humidity'] >= 20) & (all_df['humidity'] < 30),'humidity_cat'] = 2
#all_df.loc[(all_df['humidity'] >= 30) & (all_df['humidity'] < 40),'humidity_cat'] = 3
#all_df.loc[(all_df['humidity'] >= 40) & (all_df['humidity'] < 50),'humidity_cat'] = 4
#all_df.loc[(all_df['humidity'] >= 50) & (all_df['humidity'] < 60),'humidity_cat'] = 5
#all_df.loc[(all_df['humidity'] >= 60) & (all_df['humidity'] < 70),'humidity_cat'] = 6
#all_df.loc[(all_df['humidity'] >= 70) & (all_df['humidity'] < 80),'humidity_cat'] = 7
#all_df.loc[(all_df['humidity'] >= 80) & (all_df['humidity'] < 90),'humidity_cat'] = 8
#all_df.loc[(all_df['humidity'] >= 90),'humidity_cat'] = 9

all_df.loc[all_df['humidity'] < 20,'humidity_cat'] = 0
all_df.loc[(all_df['humidity'] >= 20) & (all_df['humidity'] < 40),'humidity_cat'] = 1
all_df.loc[(all_df['humidity'] >= 40) & (all_df['humidity'] < 60),'humidity_cat'] = 2
all_df.loc[(all_df['humidity'] >= 60) & (all_df['humidity'] < 80),'humidity_cat'] = 3
all_df.loc[(all_df['humidity'] >= 80),'humidity_cat'] = 4

# windspeed
all_df.loc[all_df['windspeed'] < 5,'wind_cat'] = 0
all_df.loc[(all_df['windspeed'] >= 5) & (all_df['windspeed'] < 10),'wind_cat'] = 1
all_df.loc[(all_df['windspeed'] >= 10) & (all_df['windspeed'] < 15),'wind_cat'] = 2
all_df.loc[(all_df['windspeed'] >= 15) & (all_df['windspeed'] < 20),'wind_cat'] = 3
all_df.loc[(all_df['windspeed'] >= 20) & (all_df['windspeed'] < 25),'wind_cat'] = 4
all_df.loc[(all_df['windspeed'] >= 25) & (all_df['windspeed'] < 30),'wind_cat'] = 5
all_df.loc[(all_df['windspeed'] >= 30) & (all_df['windspeed'] < 35),'wind_cat'] = 6
all_df.loc[(all_df['windspeed'] >= 35) & (all_df['windspeed'] < 40),'wind_cat'] = 7
all_df.loc[(all_df['windspeed'] >= 40) & (all_df['windspeed'] < 45),'wind_cat'] = 8
all_df.loc[(all_df['windspeed'] >= 45),'wind_cat'] = 9


In [3]:
# instead of randomly splitting our training data 
# for cross validation, let's construct a framework that's more
# in line with how the data is divvied up for this competition
# (given first 19 days of each month, what is demand for remaining days)
# so, let's split our training data into 2 time contiguous datasets
# for fitting and validating our model (days 1-14 vs. days 15-19).

# also, since submissions are evaluated based on the
# root mean squared logarithmic error (RMSLE), let's replicate
# that computation as we test and tune our model.

train_df = all_df[all_df['data_set'] == 'train']
test_df = all_df[all_df['data_set'] == 'test']

def get_rmsle(y_pred, y_actual):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

def custom_train_valid_split(data, cutoff_day=15):
    train = data[data['day'] <= cutoff_day]
    valid = data[data['day'] > cutoff_day]

    return train, valid

def prep_train_data(data, input_cols):
    X = data[input_cols].values
    y_r = data['registered_log'].values
    y_c = data['casual_log'].values

    return X, y_r, y_c

# predict on validation set & transform output back from log scale
def predict_on_validation_set(model, input_cols):
    
    train, valid = custom_train_valid_split(train_df)
    y_pred_comb_l = []
    y_actual_comb_l = []

    for year_val in [2011,2012]:
        for month_val in range(1,13):

            print(f'Now,{year_val} {month_val} training and validating...')
            # prepare training & validation set
            train_tmp = train.query('year <= @year_val and month <= @month_val')
            valid_tmp = valid.query('year == @year_val and month == @month_val')

            X_train, y_train_r, y_train_c = prep_train_data(train_tmp, input_cols)
            X_valid, y_valid_r, y_valid_c = prep_train_data(valid_tmp, input_cols)

            # training and validating
            model_r = model.fit(X_train, y_train_r)
            y_pred_r = np.exp(model_r.predict(X_valid)) - 1

            model_c = model.fit(X_train, y_train_c)
            y_pred_c = np.exp(model_c.predict(X_valid)) - 1

            y_pred_comb = np.round(y_pred_r + y_pred_c)
            y_pred_comb[y_pred_comb < 0] = 0
            y_pred_comb_l.extend(y_pred_comb)

            y_actual_comb = np.exp(y_valid_r) + np.exp(y_valid_c) - 2
            y_actual_comb_l.extend(y_actual_comb)

            #rmsle = get_rmsle(y_pred_comb, y_actual_comb)
            #rmsle_l.append(rmsle)
    
    rmsle = get_rmsle(np.array(y_pred_comb_l),np.array(y_actual_comb_l))
    
    return (np.array(y_pred_comb_l), np.array(y_actual_comb_l), rmsle)


# predict on test set & transform output back from log scale
def predict_on_test_set(model, input_cols):
    
    y_pred_comb_l = []
    for year_val in [2011,2012]:
        for month_val in range(1,13):
            
            # prepare training set
            print(f'Now,{year_val} {month_val} testing...')
            train_df_tmp = train_df.query('year <= @year_val and month <= @month_val')
            test_df_tmp = test_df.query('year == @year_val and month == @month_val')

            X_train, y_train_r, y_train_c = prep_train_data(train_df_tmp, input_cols)

            # prepare testing set
            X_test = test_df_tmp[input_cols].values
            
            model_c = model.fit(X_train, y_train_c)
            y_pred_c = np.exp(model_c.predict(X_test)) - 1

            model_r = model.fit(X_train, y_train_r)
            y_pred_r = np.exp(model_r.predict(X_test)) - 1
            
            # add casual & registered predictions together
            y_pred_comb = np.round(y_pred_r + y_pred_c)
            y_pred_comb[y_pred_comb < 0] = 0
            y_pred_comb_l.extend(y_pred_comb)

    
    return np.array(y_pred_comb_l)

In [4]:
# LightGBM for training and validation

lgb_params = {
    "objective":"regression",
    "metric": "rmse"
}

lgb_cols = [
    'weather', 'temp_cat', 'humidity', 'wind_cat',
    #'weather', 'temp', 'atemp', 'humidity', 'windspeed',
    #'weather', 'temp', 'atemp', 'humidity', 'wind_cat',
    'holiday', 'workingday', 'season',
    #'holiday', 'workingday', 'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter',
    'hour', 'dow', 'year', #'ideal', #'count_season',
    ]

train, valid = custom_train_valid_split(train_df)
y_pred_comb_l = []
y_actual_comb_l = []

for year_val in [2011,2012]:
    for month_val in range(1,13):

        print(f'Now,{year_val} {month_val} training and validating...')
        # prepare training & validation set
        train_tmp = train.query('year <= @year_val and month <= @month_val')
        valid_tmp = valid.query('year == @year_val and month == @month_val')

        X_train, y_train_r, y_train_c = prep_train_data(train_tmp, lgb_cols)
        X_valid, y_valid_r, y_valid_c = prep_train_data(valid_tmp, lgb_cols)


        train_lgb_r = lgb.Dataset(X_train, y_train_r)
        train_lgb_c = lgb.Dataset(X_train, y_train_c)
        validation_lgb_r = lgb.Dataset(X_valid, y_valid_r)
        validation_lgb_c = lgb.Dataset(X_valid, y_valid_c)

        model_r = lgb.train(
                lgb_params, train_lgb_r, 
                num_boost_round=1000, valid_sets=[train_lgb_r, validation_lgb_r], 
                verbose_eval=10, 
                early_stopping_rounds=100
                )

        model_c = lgb.train(
                lgb_params, train_lgb_c, 
                num_boost_round=1000, valid_sets=[train_lgb_c, validation_lgb_c], 
                verbose_eval=10, 
                early_stopping_rounds=100
                )

        # training and validating
        #model_r = model.fit(X_train, y_train_r)
        y_pred_r = np.exp(model_r.predict(X_valid)) - 1

        #model_c = model.fit(X_train, y_train_c)
        y_pred_c = np.exp(model_c.predict(X_valid)) - 1

        y_pred_comb = np.round(y_pred_r + y_pred_c)
        y_pred_comb[y_pred_comb < 0] = 0
        y_pred_comb_l.extend(y_pred_comb)

        y_actual_comb = np.exp(y_valid_r) + np.exp(y_valid_c) - 2
        y_actual_comb_l.extend(y_actual_comb)

        #rmsle = get_rmsle(y_pred_comb, y_actual_comb)
        #rmsle_l.append(rmsle)
    
rmsle_lgb = get_rmsle(np.array(y_pred_comb_l),np.array(y_actual_comb_l))
lgb_pred = np.array(y_pred_comb_l)
print(f'lgb_pred.shape: {np.array(y_pred_comb_l).shape}   lgb_actual.shape: {np.array(y_actual_comb_l).shape}   lgb_rmsle: {rmsle_lgb}')

#return (np.array(y_pred_comb_l), np.array(y_actual_comb_l), rmsle)

Now,2011 1 training and validating...
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015618 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 348, number of used features: 7
[LightGBM] [Info] Start training from score 3.381714
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

In [5]:
params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**params)
rf_cols = [
    #'weather', 'temp', 'atemp', 'windspeed',
    'weather', 'temp_cat', 'wind_cat',
    #'workingday', 'season', 'holiday', 'sticky',
    'workingday', 'season', 'holiday', 'humidity_cat',
    #'workingday', 'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter', 'holiday', 'sticky',
    'hour', 'dow', 'woy', 'peak'
    ]

(rf_pred, rf_actual, rf_rmsle) = predict_on_validation_set(rf_model, rf_cols)

Now,2011 1 training and validating...
Now,2011 2 training and validating...
Now,2011 3 training and validating...
Now,2011 4 training and validating...
Now,2011 5 training and validating...
Now,2011 6 training and validating...
Now,2011 7 training and validating...
Now,2011 8 training and validating...
Now,2011 9 training and validating...
Now,2011 10 training and validating...
Now,2011 11 training and validating...
Now,2011 12 training and validating...
Now,2012 1 training and validating...
Now,2012 2 training and validating...
Now,2012 3 training and validating...
Now,2012 4 training and validating...
Now,2012 5 training and validating...
Now,2012 6 training and validating...
Now,2012 7 training and validating...
Now,2012 8 training and validating...
Now,2012 9 training and validating...
Now,2012 10 training and validating...
Now,2012 11 training and validating...
Now,2012 12 training and validating...


In [6]:
print(f'rf_pred.shape: {rf_pred.shape}   rf_actual.shape: {rf_actual.shape}   rf_rmsle: {rf_rmsle}')
all_df[rf_cols].corr()

rf_pred.shape: (2286,)   rf_actual.shape: (2286,)   rf_rmsle: 0.4120389323029002


,weather,temp_cat,wind_cat,workingday,season,holiday,humidity_cat,hour,dow,woy,peak
weather,1.000000,-0.103713,0.024231,0.042061,-0.014524,0.026336,0.396231,-0.020203,-0.046424,0.009692,-0.002559
temp_cat,-0.103713,1.000000,-0.024995,0.062923,0.302664,-0.068965,-0.093425,0.134049,-0.031761,0.186577,0.139005
wind_cat,0.024231,-0.024995,1.000000,-0.000978,-0.146173,-0.005202,-0.280831,0.138555,0.000830,-0.128262,0.183375
workingday,0.042061,0.062923,-0.000978,1.000000,0.010879,-0.240847,0.011558,0.002185,-0.698028,-0.025700,-0.104563
season,-0.014524,0.302664,-0.146173,0.010879,1.000000,-0.032516,0.144388,-0.006117,-0.007448,0.814302,-0.007072
holiday,0.026336,-0.068965,-0.005202,-0.240847,-0.032516,1.000000,0.011739,0.003451,-0.169788,0.081117,0.021277
humidity_cat,0.396231,-0.093425,-0.280831,0.011558,0.144388,0.011739,1.000000,-0.269737,-0.032322,0.171397,-0.291587
hour,-0.020203,0.134049,0.138555,0.002185,-0.006117,0.003451,-0.269737,1.000000,-0.002893,-0.005437,0.488565
dow,-0.046424,-0.031761,0.000830,-0.698028,-0.007448,-0.169788,-0.032322,-0.002893,1.000000,0.009368,0.073942
woy,0.009692,0.186577,-0.128262,-0.025700,0.814302,0.081117,0.171397,-0.005437,0.009368,1.000000,-0.003243


In [7]:
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)
gbm_cols = [
    'weather', 'temp_cat', 'humidity', 'wind_cat',
    #'weather', 'temp', 'atemp', 'humidity', 'windspeed',
    #'weather', 'temp', 'atemp', 'humidity', 'wind_cat',
    'holiday', 'workingday', 'season',
    #'holiday', 'workingday', 'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter',
    'hour', 'dow', 'year', #'ideal', #'count_season',
]

(gbm_pred, gbm_actual, gbm_rmsle) = predict_on_validation_set(gbm_model, gbm_cols)

Now,2011 1 training and validating...
Now,2011 2 training and validating...
Now,2011 3 training and validating...
Now,2011 4 training and validating...
Now,2011 5 training and validating...
Now,2011 6 training and validating...
Now,2011 7 training and validating...
Now,2011 8 training and validating...
Now,2011 9 training and validating...
Now,2011 10 training and validating...
Now,2011 11 training and validating...
Now,2011 12 training and validating...
Now,2012 1 training and validating...
Now,2012 2 training and validating...
Now,2012 3 training and validating...
Now,2012 4 training and validating...
Now,2012 5 training and validating...
Now,2012 6 training and validating...
Now,2012 7 training and validating...
Now,2012 8 training and validating...
Now,2012 9 training and validating...
Now,2012 10 training and validating...
Now,2012 11 training and validating...
Now,2012 12 training and validating...


In [8]:
print(f'gbm_pred.shape: {gbm_pred.shape}   gbm_actual.shape: {gbm_actual.shape}   gbm_rmsle: {gbm_rmsle}')
all_df[gbm_cols].corr()

gbm_pred.shape: (2286,)   gbm_actual.shape: (2286,)   gbm_rmsle: 0.35051468575528655


,weather,temp_cat,humidity,wind_cat,holiday,workingday,season,hour,dow,year
weather,1.000000,-0.103713,0.418130,0.024231,0.026336,0.042061,-0.014524,-0.020203,-0.046424,-0.019157
temp_cat,-0.103713,1.000000,-0.073672,-0.024995,-0.068965,0.062923,0.302664,0.134049,-0.031761,0.038465
humidity,0.418130,-0.073672,1.000000,-0.288599,0.012676,0.014316,0.150625,-0.276498,-0.035233,-0.083546
wind_cat,0.024231,-0.024995,-0.288599,1.000000,-0.005202,-0.000978,-0.146173,0.138555,0.000830,-0.009828
holiday,0.026336,-0.068965,0.012676,-0.005202,1.000000,-0.240847,-0.032516,0.003451,-0.169788,0.029875
workingday,0.042061,0.062923,0.014316,-0.000978,-0.240847,1.000000,0.010879,0.002185,-0.698028,-0.007959
season,-0.014524,0.302664,0.150625,-0.146173,-0.032516,0.010879,1.000000,-0.006117,-0.007448,-0.010742
hour,-0.020203,0.134049,-0.276498,0.138555,0.003451,0.002185,-0.006117,1.000000,-0.002893,-0.003867
dow,-0.046424,-0.031761,-0.035233,0.000830,-0.169788,-0.698028,-0.007448,-0.002893,1.000000,0.000977
year,-0.019157,0.038465,-0.083546,-0.009828,0.029875,-0.007959,-0.010742,-0.003867,0.000977,1.000000


In [9]:
# the blend gives a better score on the leaderboard, even though it does not on the validation set
#y_pred = np.round(.4*rf_pred + .6*gbm_pred)
y_pred = np.round(.4*rf_pred + .6*lgb_pred)
print(get_rmsle(y_pred, rf_actual))

0.34215465771928516


In [10]:
# LightGBM for test and prediction
lgb_params = {
    "objective":"regression",
    "metric": "rmse"
}

y_pred_comb_l_test = []

for year_val in [2011,2012]:
    for month_val in range(1,13):

        print(f'Now,{year_val} {month_val} training and validating...')
        # prepare training & validation set
        train_df_tmp = train_df.query('year <= @year_val and month <= @month_val')
        test_df_tmp = test_df.query('year == @year_val and month == @month_val')

        X_train, y_train_r, y_train_c = prep_train_data(train_df_tmp, lgb_cols)


        train_lgb_r = lgb.Dataset(X_train, y_train_r)
        train_lgb_c = lgb.Dataset(X_train, y_train_c)

        #model_r = lgb.LGBMRegressor()
        #model_r.fit(X_train, y_train_r)
        #model_c = lgb.LGBMRegressor()
        #model_c.fit(X_train, y_train_c)
        model_r = lgb.train(
                lgb_params, train_lgb_r, 
                num_boost_round=1000, #valid_sets=[train_lgb_r, validation_lgb_r], 
                #verbose_eval=10, 
                #early_stopping_rounds=100
                )

        model_c = lgb.train(
                lgb_params, train_lgb_c, 
                num_boost_round=1000, #valid_sets=[train_lgb_c, validation_lgb_c], 
                #verbose_eval=10, 
                #early_stopping_rounds=100
                )
        
        # prepare testing set
        X_test = test_df_tmp[lgb_cols].values

        # training and validating
        #model_r = model.fit(X_train, y_train_r)
        y_pred_r = np.exp(model_r.predict(X_test)) - 1

        #model_c = model.fit(X_train, y_train_c)
        y_pred_c = np.exp(model_c.predict(X_test)) - 1

        y_pred_comb = np.round(y_pred_r + y_pred_c)
        y_pred_comb[y_pred_comb < 0] = 0
        y_pred_comb_l_test.extend(y_pred_comb)

        #y_actual_comb = np.exp(y_valid_r) + np.exp(y_valid_c) - 2
        #y_actual_comb_l.extend(y_actual_comb)

        #rmsle = get_rmsle(y_pred_comb, y_actual_comb)
        #rmsle_l.append(rmsle)
    
lgb_pred = np.array(y_pred_comb_l_test)



Now,2011 1 training and validating...
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97
[LightGBM] [Info] Number of data points in the train set: 431, number of used features: 8
[LightGBM] [Info] Start training from score 3.383360
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [11]:
rf_pred = predict_on_test_set(rf_model, rf_cols)
#gbm_pred = predict_on_test_set(gbm_model, gbm_cols)

y_pred = np.round(.4*rf_pred + .6*lgb_pred)

Now,2011 1 testing...
Now,2011 2 testing...
Now,2011 3 testing...
Now,2011 4 testing...
Now,2011 5 testing...
Now,2011 6 testing...
Now,2011 7 testing...
Now,2011 8 testing...
Now,2011 9 testing...
Now,2011 10 testing...
Now,2011 11 testing...
Now,2011 12 testing...
Now,2012 1 testing...
Now,2012 2 testing...
Now,2012 3 testing...
Now,2012 4 testing...
Now,2012 5 testing...
Now,2012 6 testing...
Now,2012 7 testing...
Now,2012 8 testing...
Now,2012 9 testing...
Now,2012 10 testing...
Now,2012 11 testing...
Now,2012 12 testing...


In [12]:
# output predictions for submission
submit_manual_blend_df = test_df[['datetime', 'count']].copy()
submit_manual_blend_df['count'] = y_pred
submit_manual_blend_df.to_csv('output/submit_manual_blend_20211006_2.csv', index=False)

In [13]:
# Level 0 RandomForestRegressor
rf_params = {'n_estimators': 1000, 'max_depth': 15, 'random_state': 0, 'min_samples_split' : 5, 'n_jobs': -1}
rf_model = RandomForestRegressor(**rf_params)
rf_cols = [
    'weather', 'temp', 'atemp', 'windspeed',
    'workingday', 'season', 'holiday', 'sticky',
    'hour', 'dow', 'woy', 'peak'
    ]
# Level 0 GradientBoostingRegressor
gbm_params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**gbm_params)
gbm_cols = [
    'weather', 'temp', 'atemp', 'humidity', 'windspeed',
    'holiday', 'workingday', 'season',
    'hour', 'dow', 'year', 'ideal', 'count_season',
]
clf_input_cols = [rf_cols, gbm_cols]
clfs = [rf_model, gbm_model]
# Create train and test sets for blending and Pre-allocate the data
blend_train = np.zeros((train_df.shape[0], len(clfs)))
blend_test = np.zeros((test_df.shape[0], len(clfs)))

In [14]:
# For each classifier, we train the classifier with its corresponding input_cols 
# and record the predictions on the train and the test set
for clf_index, (input_cols, clf) in enumerate(zip(clf_input_cols, clfs)):
    
    # prepare training and validation set
    X_train, y_train_r, y_train_c = prep_train_data(train_df, input_cols)
    
    # prepare testing set
    X_test = test_df[input_cols].values
    
    model_r = clf.fit(X_train, y_train_r)
    y_pred_train_r = np.exp(model_r.predict(X_train)) - 1
    y_pred_test_r = np.exp(model_r.predict(X_test)) - 1

    model_c = clf.fit(X_train, y_train_c)
    y_pred_train_c = np.exp(model_c.predict(X_train)) - 1
    y_pred_test_c = np.exp(model_c.predict(X_test)) - 1

    y_pred_train_comb = np.round(y_pred_train_r + y_pred_train_c)
    y_pred_train_comb[y_pred_train_comb < 0] = 0
    
    y_pred_test_comb = np.round(y_pred_test_r + y_pred_test_c)
    y_pred_test_comb[y_pred_test_comb < 0] = 0
    
    blend_train[:, clf_index] = y_pred_train_comb
    blend_test[:, clf_index] = y_pred_test_comb

In [15]:
# Level 1 Belending Classifier using LinearRegression
from sklearn.linear_model import LinearRegression
bclf = LinearRegression(fit_intercept=False)
bclf.fit(blend_train, train_df['count'])
# What is the weighted combination of the base classifiers?
print(bclf.coef_)
# Stacked and Blending predictions
y_pred_blend = np.round(bclf.predict(blend_test))
# R^2 score
bclf.score(blend_train, train_df['count'])

[0.38011939 0.66365917]


0.9676364408685162

In [16]:
# output predictions for submission
submit_stack_blend_df = test_df[['datetime', 'count']].copy()
submit_stack_blend_df['count'] = y_pred_blend
submit_stack_blend_df.to_csv('output/submit_stack_blend_20211006_2.csv', index=False)

In [17]:
all_df.head(3)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,data_set,casual_log,registered_log,count_log,date,day,month,year,hour,dow,woy,count_season,peak,ideal,sticky,temp_cat,humidity_cat,wind_cat
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,train,1.386294,2.639057,2.833213,2011-01-01,1,1,2011,0,5,52,312498,0,0,0,1.0,4.0,0.0
2011-01-01 01:00:00,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,train,2.197225,3.496508,3.713572,2011-01-01,1,1,2011,1,5,52,312498,0,0,0,1.0,4.0,0.0
2011-01-01 02:00:00,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,train,1.791759,3.332205,3.496508,2011-01-01,1,1,2011,2,5,52,312498,0,0,0,1.0,4.0,0.0
